# Recurrent Neural Networks

MUST READ ARTICLE:
https://medium.com/@sachinsoni600517/recurrent-neural-networks-rnn-from-basic-to-advanced-1da22aafa009

EXAMPLE STOCK PREDICTION
https://www.kaggle.com/code/rodsaldanha/stock-prediction-pytorch
https://medium.com/swlh/stock-price-prediction-with-pytorch-37f52ae84632

An RNN, or Recurrent Neural Network, is a type of artificial neural network designed for processing sequences of data. Unlike traditional feedforward neural networks, which process data in fixed-size input vectors, RNNs are capable of handling input sequences of variable length, making them well-suited for tasks involving time series data, natural language processing, and other sequential data types.

![alt text](https://pythongeeks.org/wp-content/uploads/2022/02/rnn-block.webp)

![alt text](https://miro.medium.com/v2/resize:fit:1400/format:webp/0*_zdmcCu76MUmw87P.png)

The key feature of RNNs is their ability to maintain a hidden state that captures information from previous time steps in the sequence. This hidden state is updated as new inputs are processed, allowing the network to capture temporal dependencies and context within the sequence.

Here's a simplified explanation of how an RNN works:

- Initialization: At the start of processing a sequence, the RNN initializes its hidden state to a fixed size vector, typically containing zeros.

- Sequential Processing: The RNN processes the input sequence one element at a time, such as one word in a sentence or one data point in a time series. At each time step, it takes the current input and combines it with the previous hidden state to produce an output and update the hidden state.

- Recurrent Connections: The recurrent connections in the RNN allow information to flow from one time step to the next, enabling the network to capture dependencies and patterns within the sequence.

RNNs have been used in various applications, including natural language processing (e.g., language modeling and machine translation), speech recognition, time series analysis, and more. However, they have some limitations, such as difficulty in capturing long-range dependencies, which has led to the development of more advanced recurrent architectures like Long Short-Term Memory (LSTM) networks and Gated Recurrent Unit (GRU) networks, which are designed to address some of these issues.

![alt text](https://miro.medium.com/v2/resize:fit:1400/1*xs2EgGPGlpWrSW4zUANYXA.png)

![alt text](https://miro.medium.com/v2/resize:fit:1194/1*B0q2ZLsUUw31eEImeVf3PQ.png)

![alt text](https://miro.medium.com/v2/resize:fit:2000/format:webp/1*ajbaACJsBtkZEq77jUQlSQ.png)

As we can see, the calculations at each time step consider the context of the previous time steps in the form of the hidden state. Being able to use this contextual information from previous inputs is the key essence to RNNs’ success in sequential problems.

While it may seem that a different RNN cell is being used at each time step in the graphics, the underlying principle of Recurrent Neural Networks is that the RNN cell is actually the exact same one and reused throughout.


## Processing RNN Outputs?

![alt text](https://cs231n.github.io/assets/rnn/types.png)

You might be wondering, which portion of the RNN do I extract my output from? This really depends on what your use case is. For example, if you’re using the RNN for a classification task, you’ll only need one final output after passing in all the input - a vector representing the class probability scores. In another case, if you’re doing text generation based on the previous character/word, you’ll need an output at every single time step.

This is where RNNs are really flexible and can adapt to your needs. As seen in the image above, your input and output size can come in different forms, yet they can still be fed into and extracted from the RNN model.

For the case where you’ll only need a single output from the whole process, getting that output can be fairly straightforward as you can easily take the output produced by the last RNN cell in the sequence. As this final output has already undergone calculations through all the previous cells, the context of all the previous inputs has been captured. This means that the final result is indeed dependent on all the previous computations and inputs.


For the second case where you’ll need output information from the intermediate time steps, this information can be taken from the hidden state produced at each step as shown in the figure above. The output produced can also be fed back into the model at the next time step if necessary.

Of course, the type of output that you can obtain from an RNN model is not limited to just these two cases. There are other methods such as Sequence-To-Sequence translation where the output is only produced in a sequence after all the input has been passed through.


![alt text](https://miro.medium.com/v2/resize:fit:1000/format:webp/1*bxfNRcWMs0xZ-3TDScqukQ.png)



## Code:

We will be building and training a basic character-level Recurrent Neural Network (RNN) to classify words. A character-level RNN reads words as a series of characters - outputting a prediction and “hidden state” at each step, feeding its previous hidden state into each next step. We take the final prediction to be the output, i.e. which class the word belongs to.

Specifically, we’ll train on a few thousand surnames from 18 languages of origin, and predict which language a name is from based on the spelling:


Dataset Link: https://download.pytorch.org/tutorial/data.zip

Included in the data/names directory are 18 text files named as [Language].txt. Each file contains a bunch of names, one name per line, mostly romanized (but we still need to convert from Unicode to ASCII).

We’ll end up with a dictionary of lists of names per language, {language: [names ...]}.

In [96]:
import random
import pandas
import os
import glob as glob
import torch
from torch import nn,optim
from string import ascii_letters
from unidecode import unidecode

In [97]:
device=torch.device("cpu")
print(device)

cpu


In [98]:
filename=os.listdir(r"D:\Datasets\RNN_data\data\names")
filename[0]

'Arabic.txt'

In [99]:
len(filename)

18

In [100]:
filename[0:19]

['Arabic.txt',
 'Chinese.txt',
 'Czech.txt',
 'Dutch.txt',
 'English.txt',
 'French.txt',
 'German.txt',
 'Greek.txt',
 'Irish.txt',
 'Italian.txt',
 'Japanese.txt',
 'Korean.txt',
 'Polish.txt',
 'Portuguese.txt',
 'Russian.txt',
 'Scottish.txt',
 'Spanish.txt',
 'Vietnamese.txt']

In [101]:
filename[0].split(".")

['Arabic', 'txt']

In [102]:
for idx,file in enumerate(filename):
    print(idx,file.split(".")[0])

0 Arabic
1 Chinese
2 Czech
3 Dutch
4 English
5 French
6 German
7 Greek
8 Irish
9 Italian
10 Japanese
11 Korean
12 Polish
13 Portuguese
14 Russian
15 Scottish
16 Spanish
17 Vietnamese


In [103]:
lang2label={
    name.split(".")[0]:torch.tensor(i,dtype=torch.float32) for i,name in enumerate(filename)
}
len(lang2label),lang2label
    

(18,
 {'Arabic': tensor(0.),
  'Chinese': tensor(1.),
  'Czech': tensor(2.),
  'Dutch': tensor(3.),
  'English': tensor(4.),
  'French': tensor(5.),
  'German': tensor(6.),
  'Greek': tensor(7.),
  'Irish': tensor(8.),
  'Italian': tensor(9.),
  'Japanese': tensor(10.),
  'Korean': tensor(11.),
  'Polish': tensor(12.),
  'Portuguese': tensor(13.),
  'Russian': tensor(14.),
  'Scottish': tensor(15.),
  'Spanish': tensor(16.),
  'Vietnamese': tensor(17.)})

In [104]:
unidecode("Ślusàrski")

'Slusarski'

In [105]:
ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [106]:
char2idx={letter: i for i,letter in enumerate(ascii_letters)}
len(char2idx),char2idx

(52,
 {'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7,
  'i': 8,
  'j': 9,
  'k': 10,
  'l': 11,
  'm': 12,
  'n': 13,
  'o': 14,
  'p': 15,
  'q': 16,
  'r': 17,
  's': 18,
  't': 19,
  'u': 20,
  'v': 21,
  'w': 22,
  'x': 23,
  'y': 24,
  'z': 25,
  'A': 26,
  'B': 27,
  'C': 28,
  'D': 29,
  'E': 30,
  'F': 31,
  'G': 32,
  'H': 33,
  'I': 34,
  'J': 35,
  'K': 36,
  'L': 37,
  'M': 38,
  'N': 39,
  'O': 40,
  'P': 41,
  'Q': 42,
  'R': 43,
  'S': 44,
  'T': 45,
  'U': 46,
  'V': 47,
  'W': 48,
  'X': 49,
  'Y': 50,
  'Z': 51})

In [107]:
vector=torch.zeros(1,52)
vector

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [108]:
v=torch.zeros(2,1,52)
v

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]]])

In [109]:
vector.shape

torch.Size([1, 52])

In [110]:
vector[0][char2idx["A"]]=1
vector

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [111]:
def name2vector(name):
    vector=torch.zeros(len(name),1,52)
    for i,names in enumerate(name):
        vector[i][0][char2idx[names]]=1
    return vector

In [112]:
name2vector("khush"),name2vector("khush").shape

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [113]:
target_lang=[]
tensor_name=[]
data_dir=r"D:\Datasets\RNN_data\data\names"
for file in os.listdir(data_dir):
    with open(os.path.join(data_dir,file)) as f:
        lang=file.split(".")[0]
        names=[unidecode(line.strip()) for line in f]
        for name in names:
            try:
                tensor_name.append(name2vector(name))
                target_lang.append(lang2label[lang])
            except KeyError:
                pass

In [114]:
len(target_lang),len(tensor_name)

(19892, 19892)

In [115]:
target_lang[2],tensor_name[2]

(tensor(0.),
 tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0.]],
 
         [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0.]],
 
         [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [116]:
len(target_lang)

19892

In [117]:
len(tensor_name)

19892

In [118]:
from sklearn.model_selection import train_test_split

In [119]:
#Here name data and language label are stored in different list and are in different forms to make sure they are corretly aligned i.e one training smple has its own output 
#we first split the indexes so the data get properly splitted with correct labels and it is safe to use it that way otherwise it may lead to wrong labels
train_idx,test_idx=train_test_split(range(len(target_lang)),test_size=0.2,shuffle=True,stratify=target_lang,random_state=42)

In [120]:
len(train_idx),len(test_idx)

(15913, 3979)

In [121]:
train_dataset=[(tensor_name[i],target_lang[i]) for i in train_idx]
test_dataset=[(tensor_name[i],target_lang[i]) for i in test_idx]
len(train_dataset),len(test_dataset)

(15913, 3979)

In [122]:
train_dataset[1][0].shape,test_dataset[1][0].shape

(torch.Size([9, 1, 52]), torch.Size([5, 1, 52]))

In [123]:
train_dataset[1][1]

tensor(14.)

In [124]:
class MyRNN(nn.Module):
    def __init__(self,hidden_size,input_size,output_size):
        super().__init__()
        self.hidden_size=hidden_size
        self.input_2_hidden=nn.Linear(
            in_features=input_size,
            out_features=hidden_size,
        )
        self.hidden_2_hidden=nn.Linear(
            in_features=hidden_size,
            out_features=hidden_size,
        )
        self.hidden_2_output=nn.Linear(
            in_features=hidden_size,
            out_features=output_size,
        )
    
    def forward(self,input,past):
        past_input_combined=nn.functional.relu(self.input_2_hidden(input)+self.hidden_2_hidden(past))
        outputs=self.hidden_2_output(past_input_combined)
        output=nn.functional.softmax(outputs,dim=1)
        return output,past_input_combined
    
    def init_hidden(self):
        return torch.zeros(1,self.hidden_size)

In [125]:
model=MyRNN(64,len(char2idx),len(lang2label))
model

MyRNN(
  (input_2_hidden): Linear(in_features=52, out_features=64, bias=True)
  (hidden_2_hidden): Linear(in_features=64, out_features=64, bias=True)
  (hidden_2_output): Linear(in_features=64, out_features=18, bias=True)
)

In [126]:
test=torch.rand(1,52)
test1=torch.rand(64,64)
test.shape
model(test,test1)

(tensor([[0.0521, 0.0488, 0.0650,  ..., 0.0387, 0.0474, 0.0582],
         [0.0500, 0.0491, 0.0630,  ..., 0.0419, 0.0481, 0.0612],
         [0.0420, 0.0542, 0.0567,  ..., 0.0390, 0.0495, 0.0587],
         ...,
         [0.0494, 0.0477, 0.0601,  ..., 0.0450, 0.0544, 0.0583],
         [0.0537, 0.0541, 0.0606,  ..., 0.0366, 0.0473, 0.0671],
         [0.0495, 0.0557, 0.0604,  ..., 0.0375, 0.0491, 0.0624]],
        grad_fn=<SoftmaxBackward0>),
 tensor([[0.0000, 0.0000, 0.1391,  ..., 0.2547, 0.5531, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.5145, 0.5568, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.2749, 0.8608, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.5272, 0.7663, 0.0000],
         [0.0000, 0.0000, 0.0861,  ..., 0.4281, 0.7161, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.1017, 0.8362, 0.0000]],
        grad_fn=<ReluBackward0>))

In [127]:
import torchinfo
from torchinfo import summary

In [128]:
summary(model,input_size=((1,52),(1,64)))

Layer (type:depth-idx)                   Output Shape              Param #
MyRNN                                    [1, 18]                   --
├─Linear: 1-1                            [1, 64]                   3,392
├─Linear: 1-2                            [1, 64]                   4,160
├─Linear: 1-3                            [1, 18]                   1,170
Total params: 8,722
Trainable params: 8,722
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.03
Estimated Total Size (MB): 0.04

In [129]:
# first index represent the id of values stored in tuple of (name,lamg), second reprsents the 1 of 1x52 and third represents the character at that index
model(train_dataset[0][0][5],model.init_hidden())

(tensor([[0.0634, 0.0636, 0.0559, 0.0567, 0.0546, 0.0528, 0.0540, 0.0636, 0.0567,
          0.0556, 0.0570, 0.0524, 0.0593, 0.0500, 0.0531, 0.0496, 0.0545, 0.0473]],
        grad_fn=<SoftmaxBackward0>),
 tensor([[0.0000, 0.1073, 0.0000, 0.1696, 0.0000, 0.0954, 0.0000, 0.0631, 0.0000,
          0.0000, 0.0924, 0.0347, 0.0056, 0.0028, 0.0000, 0.0000, 0.0000, 0.0000,
          0.1187, 0.0194, 0.0941, 0.1808, 0.0095, 0.1300, 0.0000, 0.2194, 0.0257,
          0.0975, 0.0000, 0.3151, 0.1223, 0.1194, 0.0727, 0.0602, 0.0000, 0.0056,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0743, 0.0000, 0.0000, 0.2130, 0.0000, 0.1020, 0.0000, 0.1174,
          0.0000, 0.0000, 0.0000, 0.0000, 0.1387, 0.0387, 0.0557, 0.0000, 0.0184,
          0.0000]], grad_fn=<ReluBackward0>))

In [130]:
train_dataset[1][0][5]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [131]:
loss_fn=nn.CrossEntropyLoss().to(device)
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [132]:
num_epoch=7
print_interval=500
model.to(device)
for epoch in range(num_epoch):
    for i,(name,label) in enumerate(train_dataset):
        hidden_state=model.init_hidden()
        hidden_state.to(device)
        name=name.to(device)
        label=label.long().to(device)
        for char in name:
            #char dim = 1x52
            output,hidden_state=model(char,hidden_state)
        
        loss=loss_fn(output,label.unsqueeze(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % print_interval==0:
            print(
                f"Epoch [{epoch + 1}/{num_epoch}], "
                f"Step [{i + 1}/{len(train_dataset)}], "
                #.item() to convert tensor to int
                f"Loss: {loss.item():.4f}"
            )

Epoch [1/7], Step [500/15913], Loss: 2.9815
Epoch [1/7], Step [1000/15913], Loss: 2.9814
Epoch [1/7], Step [1500/15913], Loss: 1.9815
Epoch [1/7], Step [2000/15913], Loss: 2.9445
Epoch [1/7], Step [2500/15913], Loss: 1.9815
Epoch [1/7], Step [3000/15913], Loss: 2.9686
Epoch [1/7], Step [3500/15913], Loss: 2.9630
Epoch [1/7], Step [4000/15913], Loss: 2.0084
Epoch [1/7], Step [4500/15913], Loss: 2.1030
Epoch [1/7], Step [5000/15913], Loss: 2.7692
Epoch [1/7], Step [5500/15913], Loss: 2.8363
Epoch [1/7], Step [6000/15913], Loss: 1.9815
Epoch [1/7], Step [6500/15913], Loss: 1.9972
Epoch [1/7], Step [7000/15913], Loss: 2.3234
Epoch [1/7], Step [7500/15913], Loss: 1.9815
Epoch [1/7], Step [8000/15913], Loss: 2.8809
Epoch [1/7], Step [8500/15913], Loss: 2.0352
Epoch [1/7], Step [9000/15913], Loss: 2.9781
Epoch [1/7], Step [9500/15913], Loss: 1.9815
Epoch [1/7], Step [10000/15913], Loss: 1.9815
Epoch [1/7], Step [10500/15913], Loss: 2.2731
Epoch [1/7], Step [11000/15913], Loss: 2.8061
Epoch [1

In [133]:
correct=0
with torch.inference_mode():
    for name,label in test_dataset:
        hidden_state=model.init_hidden()
        hidden_state.to(device)
        name=name.to(device)
        label=label.long().unsqueeze(0).to(device)
        for char in name:
            output,hidden_state=model(char,hidden_state)
        _,pred=torch.max(output,dim=1)
        correct+=torch.sum(pred==label)
print(f"Accuracy : {correct/len(test_dataset)*100:.4f}")

Accuracy : 73.6617


In [134]:
for lang in lang2label.items():
    print(lang)
label2lang={label.item():lang for lang,label in lang2label.items()}
label2lang

('Arabic', tensor(0.))
('Chinese', tensor(1.))
('Czech', tensor(2.))
('Dutch', tensor(3.))
('English', tensor(4.))
('French', tensor(5.))
('German', tensor(6.))
('Greek', tensor(7.))
('Irish', tensor(8.))
('Italian', tensor(9.))
('Japanese', tensor(10.))
('Korean', tensor(11.))
('Polish', tensor(12.))
('Portuguese', tensor(13.))
('Russian', tensor(14.))
('Scottish', tensor(15.))
('Spanish', tensor(16.))
('Vietnamese', tensor(17.))


{0.0: 'Arabic',
 1.0: 'Chinese',
 2.0: 'Czech',
 3.0: 'Dutch',
 4.0: 'English',
 5.0: 'French',
 6.0: 'German',
 7.0: 'Greek',
 8.0: 'Irish',
 9.0: 'Italian',
 10.0: 'Japanese',
 11.0: 'Korean',
 12.0: 'Polish',
 13.0: 'Portuguese',
 14.0: 'Russian',
 15.0: 'Scottish',
 16.0: 'Spanish',
 17.0: 'Vietnamese'}

In [135]:
label2lang={label.item():lang for lang,label in lang2label.items()}
def myrnnpredict(name):
    tensor_name=name2vector(name)
    with torch.inference_mode():
        tensor_name=tensor_name.to(device)
        hidden_state=model.init_hidden()
        hidden_state=hidden_state.to(device)
        for char in tensor_name:
            output,hidden_state=model(char,hidden_state)
        _,pred=torch.max(output,dim=1)
        model.train()
    return label2lang[pred.item()]

In [148]:
myrnnpredict("zhang")

'Russian'